# Working with many and/or large files

In this section, we will take a look at techniques for working with many files, as well as large files.

In [216]:
import pandas as pd
from dfply import *
from functools import reduce

## Baseball data

We will be using the [Baseball Databank](https://github.com/chadwickbureau/baseballdatabank), make sure you have these data cloned into `./data/baseball`.

In [50]:
!git clone https://github.com/chadwickbureau/baseballdatabank.git ./data/baseball

fatal: destination path './data/baseball' already exists and is not an empty directory.


## Working with many files.

* Use `glob.glob` to find all files that match a pattern
* Convert all files to `pd.DataFrames`
* Store the `df` in a list or dictionary

## What the heck is a `glob`

`glob.glob`

* Takes a path regular expression
* Returns a list of files that match the patterm
* Relative paths!

## Store in `dict` or `list`?

* Natural sequence/order? $\rightarrow$ `list`
    *  Example: Lakes data and years are a natural sequence
* Easier to refer by name? $\rightarrow$ `dict`
    * Baseball files have no order and easier to refer to by name

## Example 1 - Reading the baseball database.

#### Step 1 - Get the files names

In [154]:
from glob import glob
files = glob('./data/baseball/core/*.csv')
files

['./data/baseball/core/AwardsManagers.csv',
 './data/baseball/core/Managers.csv',
 './data/baseball/core/AwardsPlayers.csv',
 './data/baseball/core/Fielding.csv',
 './data/baseball/core/Salaries.csv',
 './data/baseball/core/Parks.csv',
 './data/baseball/core/Schools.csv',
 './data/baseball/core/People.csv',
 './data/baseball/core/PitchingPost.csv',
 './data/baseball/core/Teams.csv',
 './data/baseball/core/Appearances.csv',
 './data/baseball/core/AwardsSharePlayers.csv',
 './data/baseball/core/TeamsFranchises.csv',
 './data/baseball/core/Batting.csv',
 './data/baseball/core/ManagersHalf.csv',
 './data/baseball/core/FieldingOF.csv',
 './data/baseball/core/Pitching.csv',
 './data/baseball/core/CollegePlaying.csv',
 './data/baseball/core/HomeGames.csv',
 './data/baseball/core/HallOfFame.csv',
 './data/baseball/core/AwardsShareManagers.csv',
 './data/baseball/core/BattingPost.csv',
 './data/baseball/core/TeamsHalf.csv',
 './data/baseball/core/SeriesPost.csv',
 './data/baseball/core/Fielding

#### Step 2 - Make helper functions to get the name from path

In [155]:
import re
FILE_NAME_RE = re.compile(r'^\./data/baseball/core/([a-zA-Z_]*)\.csv$')
file_name = lambda p: FILE_NAME_RE.match(p).group(1) 
file_names = lambda files: [file_name(p) for p in files]
file_names(files)

['AwardsManagers',
 'Managers',
 'AwardsPlayers',
 'Fielding',
 'Salaries',
 'Parks',
 'Schools',
 'People',
 'PitchingPost',
 'Teams',
 'Appearances',
 'AwardsSharePlayers',
 'TeamsFranchises',
 'Batting',
 'ManagersHalf',
 'FieldingOF',
 'Pitching',
 'CollegePlaying',
 'HomeGames',
 'HallOfFame',
 'AwardsShareManagers',
 'BattingPost',
 'TeamsHalf',
 'SeriesPost',
 'FieldingPost',
 'AllstarFull',
 'FieldingOFsplit']

#### Step 3 - Use a comprehension to read in all files

**Note:** The data is small (< 10mb total) so it is safe to read all at once.

In [53]:
dfs = {name:pd.read_csv(path) for name, path in zip(file_names(files), files)}
dfs['Pitching'].head()

,playerID,yearID,stint,teamID,lgID,W,L,G,GS,CG,...,IBB,WP,HBP,BK,BFP,GF,R,SH,SF,GIDP
0,bechtge01,1871,1,PH1,NaN,1,2,3,3,2,...,NaN,7,NaN,0,146.0,0,42,NaN,NaN,NaN
1,brainas01,1871,1,WS3,NaN,12,15,30,30,30,...,NaN,7,NaN,0,1291.0,0,292,NaN,NaN,NaN
2,fergubo01,1871,1,NY2,NaN,0,0,1,0,0,...,NaN,2,NaN,0,14.0,0,9,NaN,NaN,NaN
3,fishech01,1871,1,RC1,NaN,4,16,24,24,22,...,NaN,20,NaN,0,1080.0,1,257,NaN,NaN,NaN
4,fleetfr01,1871,1,NY2,NaN,0,1,1,1,1,...,NaN,0,NaN,0,57.0,0,21,NaN,NaN,NaN


## <font color="red"> Exercise 1 </font>

Use `glob` to read the following files into a `dict`: `Person.csv`, `Survey.csv`, `Site.csv`, `Visited.csv`

In [157]:
# Your code here
files = glob('./data/*.csv')
files

['./data/Site.csv',
 './data/heroes_information.csv',
 './data/uber-raw-data-apr14.csv',
 './data/TB_burden_age_sex_2019-01-07.csv',
 './data/Visited.csv',
 './data/PEW_income_religion.csv',
 './data/health_survey.csv',
 './data/Artists.csv',
 './data/ebola_data_db_format.csv',
 './data/Artworks.csv',
 './data/super_hero_powers.csv',
 './data/TB_example.csv',
 './data/OralFlag_example.csv',
 './data/Person.csv',
 './data/TB_bad.csv',
 './data/Survey.csv']

In [215]:
FILE_NAME_RES = re.compile(r'^\./data/(Site|Person|Visited|Survey)\.csv$')
file_name = lambda p: FILE_NAME_RES.findall(p)
file_names = lambda files: [file_name(p) for p in files if file_name(p) != []]
file_names(files)


[['Site'], ['Visited'], ['Person'], ['Survey']]

In [220]:
filename = reduce(lambda x,y: x+y,file_names(files))
filename

['Site', 'Visited', 'Person', 'Survey']

In [224]:
dfa = {name:pd.read_csv(path) for name, path in zip(filename, files)}
dfa['Survey'].head()


,country,iso2,iso3,iso_numeric,year,measure,unit,age_group,sex,best,lo,hi
0,Afghanistan,AF,AFG,4,2017,inc,num,0-14,a,7400,6500.0,8400.0
1,Afghanistan,AF,AFG,4,2017,inc,num,0-14,f,3500,3200.0,3900.0
2,Afghanistan,AF,AFG,4,2017,inc,num,0-14,m,3900,3500.0,4300.0
3,Afghanistan,AF,AFG,4,2017,inc,num,0-4,f,1800,1700.0,1900.0
4,Afghanistan,AF,AFG,4,2017,inc,num,0-4,m,2200,2000.0,2300.0


## Up Next

In [Lecture 3.2 - Aggregating Large Files with Pandas](./3_2_aggregating_large_files_in_pandas.ipynb), we will look at using `pandas` to read and aggregate chunks of a large file.